In [47]:
# Importing required Packages
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.integrate import cumtrapz

ImportError: cannot import name 'cumtrapz' from 'scipy.integrate' (C:\Users\trent\OneDrive\Desktop\EE361\.venv\Lib\site-packages\scipy\integrate\__init__.py)

In [48]:
# Changing directory to Waveform location
%cd C:\Users\trent\Desktop\ML Paper -02 Data\Vivek_2022\CF_25_CCS_0-6

[WinError 3] The system cannot find the path specified: 'C:\\Users\\trent\\Desktop\\ML Paper -02 Data\\Vivek_2022\\CF_25_CCS_0-6'
C:\Users\trent\PycharmProjects\EE491


In [6]:
# Reading calibration data to set boundaries on graphical interface
calibration = np.loadtxt("Filtered Waveform_Pressure_SampleCalibration.txt")
# Splitting columns into time and amplitude
time_cal = calibration[:,0]
amplitude_cal = calibration[:,1]

In [7]:
# Setting window index's

cal_w1 = 1061
cal_w2 = 1163
samp_w1 = 1300
samp_w2 = 2000
proc_W1 = 111
proc_W2 = 331

In [8]:
# Filtered Waveform Processing

# Creating finalized metric of each filtered waveform
wf_metric = np.zeros(35)
plt.figure(figsize=(12,10))
# creating storage for normalized fast fourier transform (FFT) outputs
norm_FFT = np.zeros((2251, 35))

<Figure size 1200x1000 with 0 Axes>

In [9]:
# Loading all filtered samples into previously allocated metrics
for i in range(35):
    # changing waveform number in according to loop
    wf_1 = np.loadtxt(f'Filtered Waveform_Pressure_Sample{i+1}.txt')
    # cropping sample and calibration waveforms
    y_cal_filt = calibration[:,i]
    y_samp_filt = wf_1[:,1]
    y_filt_cal = y_cal_filt[cal_w1:cal_w2]
    y_filt_samp = y_samp_filt[samp_w1:samp_w2]
    # Applying FFT to analyze freq domain by taking absolute value for amplitude spectrum
    ampspec_cal = np.abs(fft(y_filt_cal, 11000))
    ampspec_samp = np.abs(fft(y_filt_samp, 11000))
    # zooming to fit on amplitude outputs
    len_cal = len(ampspec_cal)
    len_samp = len(ampspec_samp)
    # adjusting sample and calibration frequency axis's
    freq_axis_cal = 1. / (wf_1[1, 0] - wf_1[0, 0]) * np.arange(len_cal / 2 + 1) / len_cal
    freq_axis_samp = 1. / (wf_1[1, 0] - wf_1[0, 0]) * np.arange(len_samp / 2 + 1) / len_samp
    # attenuation coefficient, tells how weakened signal is when passing though our capsules
    att_coef = (1 / (1j * 5.55e-3)) * np.log(ampspec_samp / ampspec_cal)
    # imaginary part to keep relevance in our matrix
    imag_att = np.imag(att_coef)
    # truncation of arrays
    ampspec_cal = ampspec_cal[:int(len_cal/2+1)]
    imag_att = imag_att[:int(len_samp/2+1)]
    # corrections to remove trends downward or upward upon compresion. makes signal flat for modeling
    y_N = ((imag_att[331] - imag_att[111]) / (freq_axis_samp[331] - freq_axis_samp[111])) * freq_axis_samp + imag_att[111] - (
            (imag_att[331] - imag_att[111]) / (freq_axis_samp[331] - freq_axis_samp[111])) * freq_axis_samp[111]
    y_N_T = y_N.T
    imag_att_01 = imag_att - y_N_T
    # signal processing slicing. subtracts mean to center signal on y = 0
    y_filt_samp_temp = imag_att_01[proc_W1:proc_W2]
    y_filt_samp_01 = y_filt_samp_temp - np.mean(y_filt_samp_temp)
    # Using FFT again to extract features of filtered section and zoom on area
    ampspec_samp_final = np.abs(fft(y_filt_samp_01), 4500)
    len_ampspec_samp = len(ampspec_samp_final)
    # frequency axis calibration for new filtered sample wave form
    freq_a_1 = 1. / (freq_axis_samp[1] - freq_axis_samp[0]) * np.arange(len_ampspec_samp / 2 + 1) / len_ampspec_samp
    # normalizing wave form from 0 to 1 range for easier modeling
    ampspec_samp_final = (ampspec_samp_final[:int(len_ampspec_samp / 2 + 1)] - np.min(ampspec_samp_final[:int(len_ampspec_samp / 2 + 1)])) / (
        np.max(ampspec_samp_final[:int(len_ampspec_samp / 2 + 1)]) - np.min(ampspec_samp_final[:int(len_ampspec_samp / 2 + 1)]))
    norm_FFT[:,i] = ampspec_samp_final
    # plotting normalized and processed FFT of each waveform
    plt.plot(freq_a_1 * 10 ** 6, ampspec_samp_final, 'k--', linewidth=1)
    # plot modifications and labeling
    plt.xlim([0, 20])
    plt.ylim([-.1, 1.1])
    plt.xlabel('f(MHz)', fontsize=16)
    plt.ylabel('Amplitude', fontsize=16)
    plt.grid(True)
    plt.box(True)
    # extraction of area under curve features.
    Int = np.concatenate(([0], cumtrapz(freq_axis_samp * 10 ** -6, imag_att_01 / (3 - 1))))
    Intv = lambda a, b: np.max(Int[freq_axis_samp * 10 ** -6 <= b]) - np.min(Int[freq_axis_samp * 10 ** -6 >= a])
    wf_metric[i] = Intv(1, 3)

# final plotting / display of plot
plt.show()
norm_FFT[2250,:] = 1
